# Transforming the Meta-DataFrame into a Neo4j Event Log Class Diagram Graph
The following script demonstrates how the Meta-DataFrame can be modelled in Neo4j as an Event Log Class Diagram Graph of a given event log.

In [9]:
# import the libraries
import json
import pandas as pd
import numpy
import re #regular expressions for working with text data, especially extracting information from a piece of text

In [10]:
# Change the title names to circumvent certain Neo4j whitespace issues. The index_to_neo4j dictionary  maps keys will map the values from the Meta-DataFrame to the Neo4j graph later
index_to_neo4j = {
    'Title': 'title',
    'Data_type': 'data_type',
    'Number_of_entries': 'num_entries',
    'Number_of_unique_entries': 'num_unique_entries',
    'Number_of_duplicate_entries':'num_duplicate_entries',
    'Number_of_undefined_entries': 'num_undefined_entries',
    'Percentage_of_undefined_entries': 'percentage_undefined_entries',
}
index_to_neo4j

{'Title': 'title',
 'Data_type': 'data_type',
 'Number_of_entries': 'num_entries',
 'Number_of_unique_entries': 'num_unique_entries',
 'Number_of_duplicate_entries': 'num_duplicate_entries',
 'Number_of_undefined_entries': 'num_undefined_entries',
 'Percentage_of_undefined_entries': 'percentage_undefined_entries'}

In [11]:
#Define the Meta-DataFrame and generate the first command to clear any related event data in Neo4j
event_log_name = 'BPI_C_2019_2_final_synthetic' #change accordingly for each EL
print('// Delete all nodes and their relationships:')
print(f'MATCH (node:{event_log_name}) DETACH DELETE node;')
print()

// Delete all nodes and their relationships:
MATCH (node:BPI_C_2019_2_final_synthetic) DETACH DELETE node;



In [12]:
#Import the event log Meta-DataFrame based on event_log_name above
overview_table = pd.read_csv(f'analysis/overview_{event_log_name}.csv', index_col = 'Property') #use Property as the index_column
overview_table = overview_table.T #switch table layout
overview_table

Property,caseID,timestamp,event activity,event_ID,case Spend type,Company by Case,C_Document Type,Case Sub spend text,C_Purchasing Document,case Purch. Doc. Category name,...,case Spend classification text,case Source,case Name,CASE GR,case Item,case concept:name,case Goods Receipt,User,Resource,event Net Worth (CHF)
Unnamed: 0,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,...,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier,BPI_C_2019_2nd_Supplier
Title,Case ID,Timestamp,Activity,Event ID,case Spend type,Company by Case,C_Document Type,Case Sub spend text,C_Purchasing Document,case Purch. Doc. Category name,...,case Spend classification text,case Source,case Name,CASE GR,case Item,case concept:name,case Goods Receipt,User,Resource,event Net Worth (CHF)
Importance,Mandatory attribute,Mandatory attribute,Mandatory attribute,Mandatory attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,...,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute
Level,Case level,Event level,Event level,Event level,Case level,Case level,Case level,Case level,Case level,Case level,...,Case level,Case level,Case level,Case level,Case level,Case level,Case level,Event level,Event level,Event level
Data type,string,"datetime64[ns, UTC]",string,float64,string,string,string,string,int64,string,...,string,string,string,int64,int64,string,int64,string,string,float64
Number of entries,349525,349525,349525,349525,349525,349525,349525,349525,349525,349525,...,349525,349525,349525,349525,349525,349525,349525,349525,349525,349525
Number of unique entries,100632,31,37,1804,20,3,3,131,30584,1,...,3,1,1420,2,317,100632,2,339,339,14542
List up to 100 unique entries to view the structure of the event attributes,"['2000013534 2', '2000013533 2', '4507021815 1...","[Timestamp('1970-01-01 00:00:10+0000', tz='UTC...","['SRM: Document Completed', 'SRM: Complete', '...","[189000000000000.0, 314000000000000.0, 3010000...","['Others', 'CAPEX & SOCS', 'Additives', 'Sales...","['Company_0', 'Company_3', 'Company_1']","['PO EC ', 'Standard Purchase Order', 'Framewo...","['Raw Material', 'Laboratory Supplies & Servic...","[2000013534, 2000013533, 4507021815, 200001353...",['Purchase order'],...,"['OTHER', 'NPR', 'PR', <NA>]",['Source_SystemID_001'],"['vendor_0027', 'vendor_0028', 'vendor_0164', ...","[0, 1]","[2, 10, 20, 30, 1, 40, 60, 180, 50, 130, 70, 8...","['2000013534_00002', '2000013533_00002', '4507...","[1, 0]","['batch_00', 'user_147', 'user_019', 'user_029...","['batch_00', 'user_147', 'user_019', 'user_029...","[179.0, 273.0, 1416.0, 9.0, 3073.0, 4330.0, 82..."
Length of the list of up to 100 unique entries,100,31,37,100,21,3,3,100,100,1,...,4,1,100,2,100,100,2,100,100,100


In [13]:
# Gather overview_columns
overview_columns = overview_table.columns

#View
overview_columns


Index(['caseID', 'timestamp', 'event activity', 'event_ID ', 'case Spend type',
       'Company by Case', 'C_Document Type', 'Case Sub spend  text',
       'C_Purchasing Document', 'case Purch. Doc. Category name',
       'case Vendor', 'Type of case Item ', 'case Item Category',
       'case Spend classification text', 'case Source', 'case Name', 'CASE GR',
       'case Item', 'case concept:name', 'case Goods Receipt', 'User',
       'Resource', 'event Net Worth (CHF)'],
      dtype='object', name='Property')

<h1>Creating the Neo4j Commands</h1>
The author hardcoded the Neo4j commands as changes could be made easily - when adjustments have to be made to the Neo4jDB instance.

In [14]:
# create a two lists
neo4j_lines = []
mandatory_columns = []

for overview_column in overview_columns:
  spalte = overview_table[overview_column]
  spalte_json_string = spalte.to_json(default_handler=str)
  neo4j_properties = json.loads(spalte_json_string)

  overview_level = neo4j_properties['Level']

  property_label_identifier = neo4j_properties['Importance']
  if property_label_identifier == 'Mandatory attribute':
    mandatory_columns.append(overview_column)

  properties_list_of_strings = [f'`{key}`: "{value}"' for (key, value) in neo4j_properties.items()]

  neo4j_create_command = ''
  neo4j_create_command += f'''CREATE (`{overview_column}`:`{overview_column}`:`{overview_level}`:`{property_label_identifier}`:{event_log_name} '''
  neo4j_create_command += f'''{{name: '{overview_column}', {', '.join(properties_list_of_strings)}}})'''

  neo4j_lines.append(neo4j_create_command)

neo4j_lines.append('CREATE (`caseID`) - [:CASE_TO_EVENT] -> (`event_ID `)')
neo4j_lines.append('CREATE (`event_ID `) - [:EVENT_RELATIONSHIP] -> (`event activity`)')
neo4j_lines.append('CREATE (`event_ID `) - [:EVENT_RELATIONSHIP] -> (`timestamp`)')

for overview_column in overview_columns:
  if overview_column not in mandatory_columns and 'event' in overview_column:
    neo4j_create_command = f'''CREATE (`event_ID `) - [:EVENT_RELATIONSHIP] -> (`{overview_column}`)'''
    neo4j_lines.append(neo4j_create_command)
  if overview_column not in mandatory_columns and 'case' in overview_column:
    neo4j_create_command = f'''CREATE (`caseID`) - [:CASE_RELATIONSHIP] -> (`{overview_column}`)'''
    neo4j_lines.append(neo4j_create_command)

# To add the preceding sequence of Cypher CREATE commands.
neo4j_lines.append(';')

neo4j_command = '\n'.join(neo4j_lines)

neo4j_command = '// Create all nodes and their relationships\n' + neo4j_command

with open(f'analysis/overview_{event_log_name}.cypher', 'w') as overview_file:
  print(neo4j_command, file=overview_file)

In [15]:
print('// Show all nodes and their relationships:')
print(f'MATCH (event_log:{event_log_name}) RETURN event_log;')
print()

// Show all nodes and their relationships:
MATCH (event_log:BPI_C_2019_2_final_synthetic) RETURN event_log;



In [16]:
# For dataset 2 the commands have to be manually changed to create the Neo4j commands
# The case table is an exploratory acitivty to model the longest case to visualize all event contained in that case in Neo4j commands
case_table = pd.read_csv(f'analysis/longest_case_in_{event_log_name}.csv', index_col='EventIndex')
case_table

# The index of a case table is the index of an event.
# The case table is an event table that only contains events belonging to a certain case.

cypher_code = 'CREATE\n'
for event_index in case_table.index:
  event = case_table.loc[event_index]
  cypher_code += \
    f'''  (event_{event_index}:`{event['event activity']}` {{\n''' +\
    f'''    `caseID`: "{event['caseID']}",\n''' +\
    f'''    `timestamp`: "{event['timestamp']}",\n''' +\
    f'''    `event activity`: "{event['event activity']}",\n''' +\
    f'''    `event_ID `: "{event['event_ID ']}",\n''' +\
    f'''    `case Spend type`: "{event['case Spend type']}",\n''' +\
    f'''    `Company by Case`: "{event['Company by Case']}",\n''' +\
    f'''    `C_Document Type`: "{event['C_Document Type']}",\n''' +\
    f'''    `Case Sub spend  text`: "{event['Case Sub spend  text']}",\n''' +\
    f'''    `C_Purchasing Document`: "{event['C_Purchasing Document']}",\n''' +\
    f'''    `case Purch. Doc. Category name`: "{event['case Purch. Doc. Category name']}",\n''' +\
    f'''    `case Vendor`: "{event['case Vendor']}",\n''' +\
    f'''    `Type of case Item `: "{event['Type of case Item ']}",\n''' +\
    f'''    `case Item Category`: "{event['case Item Category']}",\n''' +\
    f'''    `case Spend classification text`: "{event['case Spend classification text']}",\n''' +\
    f'''    `case Source`: "{event['case Source']}",\n''' +\
    f'''    `case Name`: "{event['case Name']}",\n''' +\
    f'''    `CASE GR`: "{event['CASE GR']}",\n''' +\
    f'''    `case Item`: "{event['case Item']}",\n''' +\
    f'''    `case concept:name`: "{event['case concept:name']}",\n''' +\
    f'''    `case Goods Receipt`: "{event['case Goods Receipt']}",\n''' +\
    f'''    `User`: "{event['User']}",\n''' +\
    f'''    `Resource`: "{event['Resource']}",\n''' +\
    f'''    `event Net Worth (CHF)`: "{event['event Net Worth (CHF)']}"\n''' +\
    f'''  }}),\n'''

for first_event_index, second_event_index in zip(case_table.index, case_table.index[1:]):
  first_event = case_table.loc[first_event_index]
  second_event = case_table.loc[second_event_index]
  cypher_code += f'''  (event_{first_event_index})-[:DIRECTLY_FOLLOWS]->(event_{second_event_index}),\n'''

# Delete the last comma symbol occurrence.
cypher_code = cypher_code[:-2]
cypher_code += '\n;'

with open(f'analysis/longest_case_in_{event_log_name}.cypher', 'w') as case_file:
  print(cypher_code, file=case_file)

# End of Script 2 Transforming the Meta-DataFrame into a Neo4j Event Log Class Diagram Graph
Author: Kyle Smith <br>
Script: For Masterthesis <br>
University of Camerino & University of Applied Sciences Northwestern Switzerland